# Week5 - Learn geopandas by plotting tornados on a map

In this tutorial we will take a look at the powerful *geopandas* library and use it to plot historical tornado data on a map of the United States.

### Here are the commands you will need to run if have not already install geopandas
#### Prerequisites 
```
conda install geopandas
conda install -c conda-forge descartes
conda update geopandas

or try
pip install geopandas
pip install --upgrade geopandas
pip install --upgrade fiona

```

The data we will be working with comes from the US Census and is in a common shapefile format.

In [ ]:
ls -al data

### Import geopandas and load the U.S. map shapefile

In [ ]:
import matplotlib.pyplot as plt
import geopandas

states = geopandas.read_file('data/usa-states-census-2014.shp')

In [ ]:
states.head()

### Reproject coordinates

By default this shp file is in WGS 84.

In [ ]:
states.crs #Coordinate Reference System

To make the map look a little more familiar lets reproject it's coordinates to **Mercator**.

geopandas requires we know the geospatial reference system identifier so here is a list of common ones.

* **"EPSG:4326"** WGS84 Latitude/Longitude, used in GPS
* **"EPSG:3395"** Spherical Mercator. Google Maps, OpenStreetMap, Bing Maps
* **"EPSG:32633"** UTM Zones (North) - (Universal Transverse Mercator)
* **"EPSG:32733"** UTM Zones (South) - (Universal Transverse Mercator)

In [ ]:
states = states.to_crs("EPSG:3395")

Now lets plot the U.S. map and see what we get

In [ ]:
states.plot()

We can also plot the state polygons with no fill color by using boundary

In [ ]:
states.boundary.plot()

In [ ]:
west = states[states['region'] == 'West']
west

In [ ]:
west = states[states['region'] == 'West']

west.plot(cmap='Pastel2', figsize=(12, 12))

### Add outlines and labels to each State

Here is another plot of the U.S West but this time we are going to use a lambda function to plot the state name over each state.  We also plotting the state shapes with a black outline.

As a bonus code snippet,  I have added a vertical watermark to the left side of the image.

In [ ]:
fig = plt.figure(1, figsize=(25,15)) 
ax = fig.add_subplot()

west.apply(lambda x: ax.annotate(x.NAME, xy=x.geometry.centroid.coords[0], ha='center', fontsize=14), axis=1)

west.boundary.plot(ax=ax, color='Red', linewidth=.6)

west.plot(ax=ax, cmap='Pastel2', figsize=(12, 12))

ax.text(-0.05, 0.5, 'West Map', transform=ax.transAxes,
        fontsize=20, color='gray', alpha=0.5,
        ha='center', va='center', rotation=90)

### Load the NOAA US tornado dataset 

Source: https://www.spc.noaa.gov/gis/svrgis/

One of the cool features of geopandas is that you can load a shapefile directly from a zip archive without first unzipping it.  The file path to open a zipped shapefile is prefixed with `zip://`

However, there is a caveat.  If the zip archive contains a subfolder you must specify it by appending `!foldername` to the path.

In [ ]:
ls -al  data

### Load the shapefile from zip archive

We will use the above mentioned technique to load the `1950-2018-torn-initpoint.shp` shapefile from the zip archive.

In [ ]:
tornados = geopandas.read_file('zip://data/1950-2018-torn-initpoint.zip!1950-2018-torn-initpoint')

<font color="green">
Note: If loading from inside a zip file confuses you don't let is slow you down, just unzip the archive.
</font>

Lets have a look at the DataFrame

In [ ]:
tornados.shape

In [ ]:
tornados.info()

In [ ]:
tornados.head(10)

#### Reproject coordinates

We will also need to reproject it's coordinates to **Mercator** so that our maps line up.

In [ ]:
tornados = tornados.to_crs("EPSG:3395")

Take note how reprojecting our coordinates to Mercator moves the decimal point in our lat, long data.  This will come into play later.

In [ ]:
tornados[:1].geometry

### Our first plot of U.S. tornado data

In [ ]:
tornados.plot( figsize=(12,9), color='red', marker='x', markersize=1)

### Now lets plot the tornados data on top of the U.S. basemap

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(12,9)) 
ax = fig.add_subplot()
states.boundary.plot(ax=ax, color='black', linewidth=.8)

tornados.plot(ax=ax, color='red', marker='.', markersize=1)

### Great!

We even chose an ominous red v-shaped marker to represent the tornados.  The two maps line up perfectly but it looks like we have some outliers data for Hawaii, Alaska, and Puerto Rico that are stretching out our map canvas.

We want to focus on the lower 48 states so now we need to narrow down our view.  We can accomplish this in a couple of ways. One way is to remove any tornados from our dataframe that are not in the lower 48.  Another approach is to simply limit the viewport area of the map with `ax.set_xlim()` and `ax.set_ylim()`.

Our first thought might be to pass in typical lat,long data such as `ax.set_xlim(-101.12345, 72.12345)`.  After we reprojected our maps to Mercator our coordinates data now look like this `ax.set_xlim(-10112345, 7212345)`.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(12,9)) 
ax = fig.add_subplot()

# US Lower 48 Bounding Box
# -141.00000, 26.00000, -65.50000, 72.00000

ax.set_xlim(-14100244, -7200000)
ax.set_ylim(2600000, 6550000)

fig.suptitle('<Yourname>+United States Tornado Map (1950-2018)', fontsize=16)

states.boundary.plot(ax=ax, color='black', linewidth=.8)

tornados.plot(ax=ax, color='red', marker='v', markersize=8)
ax.text(-0.05, 0.5, 'Tornado', transform=ax.transAxes,
        fontsize=20, color='black', alpha=0.5,
        ha='center', va='center', rotation=90)

Hey! That was a success, now let's press on.

### Get total tornados by State

I already know the answer to this question (because its my home) but...

Which state do you think has had the most tornados?

> *Hint: In Texas we call them twisters*

In [ ]:
# Create a copy of the original DataFrame
twisters_by_state = tornados.copy()
# Add a new column and set the value to 1
twisters_by_state['tornados'] = 1

# use groupby() and count() to total up all the tornadoes by state
twisters_by_state = twisters_by_state[['st','tornados']].groupby('st').count()

# sort by most tornadoes first
twisters_by_state.sort_values('tornados', ascending=False).head(10)

Yep, Texas has the most tornados! 

We can also easily plot this data by calling plot()

In [ ]:
twisters_by_state.plot.bar(figsize=(12,6), title='Tornados by State (1950-2018)')

Notice that our sorting did not stick, this is because we did not reassign the DataFrame only output it to the screen.

We can chain in a call to `.sort_values()` to get our ordering to work in the plot.

I will also select the top 10 be adding `[:10]` which means "slice first 10 rows".

Now we can see our state labels a little better.

In [ ]:
twisters_by_state.sort_values('tornados', ascending=False)[:10].plot.bar(figsize=(12,6), title='Tornados by State (Top 10)')

If we just want a list of the top 10 states we can do something like this.

In [ ]:
top10 = twisters_by_state.sort_values('tornados', ascending=False)[:10].index
top10

### Plot tornados by state

Now that we know that Texas has had the most tornados lets subset our tornado dataset to only show ***Texas Twisters***.

In [ ]:
texas_map = states[states['NAME'] == 'Texas']
texas_map.plot()

In [ ]:
texas_twisters = tornados[tornados['st'] == 'TX']

texas_twisters

In [ ]:
texas_twisters.plot(markersize=1)

Now that we have both of our DataFrames `texas_map` and `texas_twisters` lets overlay combine and plot them.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(10,10)) 
ax = fig.add_subplot()

# Adjust bounding box to only Texas
# -119.00000, 29.30000, -103.80000, 44.00000

ax.set_xlim(-11910000, -10380000)
ax.set_ylim(2930000, 4400000)

fig.suptitle('<Yourname>+Texas Tornados (1950-2018)', fontsize=16)

texas_map.boundary.plot(ax=ax, color='black', linewidth=.8)

texas_twisters.plot(ax=ax, color='red', marker='v', markersize=20)

Now lets move over to the path shapefile and see what we have to work with.

### Load the NOAA US tornado paths dataset 

In [ ]:
tornado_paths = geopandas.read_file('zip://data/1950-2018-torn-aspath.zip!1950-2018-torn-aspath')

In [ ]:
tornado_paths.head()

Reproject to Mercator

In [ ]:
tornado_paths = tornado_paths.to_crs("EPSG:3395")

In [ ]:
tornado_paths.plot(figsize=(14,8))

In [ ]:
texas_twister_paths = tornado_paths[tornado_paths['st'] == 'TX']
texas_twister_paths.head()

### Plot tornado points and paths for Texas

Now that we have our tornado paths DataFrame narrowed down to Texas lets plot the paths.

For this next map I will plot the start point of each Tornado as pink and the path data as Red.

As you can see, path data does not exist for all recorded tornados.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(10,10)) 
ax = fig.add_subplot()

# Adjust bounding box to only Texas
# -119.00000, 29.30000, -103.80000, 44.00000

ax.set_xlim(-11910000, -10380000)
ax.set_ylim(2930000, 4400000)

fig.suptitle('<Yourname>+Texas Tornado Paths (1950-2018)', fontsize=16)

texas_map.boundary.plot(ax=ax, color='black', linewidth=.8)
texas_twisters.plot(ax=ax, color='pink', marker='v', markersize=10)
texas_twister_paths.plot(ax=ax, color='red')

### Color coded tornados by year

In this variation of the Texas tornado map we are color coding the tornado markers and paths by year with a colormap file.  To do this we add the parameter `column='yr'` to the call to `plot()`.

The colormap used `cmap='coolwarm'` allows us to see older tornado records as cool blue colors and more recent tornados as warm red colors.  Note that we also removed the `color` parameter as cmap and color cannot be used simultaneously.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(10,10)) 
ax = fig.add_subplot()

# Adjust bounding box to only Texas
# -119.00000, 29.30000, -103.80000, 44.00000

ax.set_xlim(-11910000, -10380000)
ax.set_ylim(2930000, 4400000)

fig.suptitle('<Yourname>+Texas Tornado Paths (1950-2018)', fontsize=16)

texas_map.boundary.plot(ax=ax, color='black', linewidth=.8)
texas_twisters.plot(ax=ax, column='yr', cmap="coolwarm", marker='v', markersize=10)
texas_twister_paths.plot(ax=ax, column='yr', cmap="coolwarm")

Congratulations, you are well on your way to becoming a GIS expert with `geopandas`. 